In [ ]:
from feature_extraction.orb import ORB
import os
import cv2
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
# from feature_extraction.HOG import HOG

DATA_PATH="Caltech20"
TRAINING_DIR=os.path.join(DATA_PATH, "training")
TEST_DIR=os.path.join(DATA_PATH, "testing")

classes = []
features = {}
all_descriptor = []

from skimage.feature import hog
from skimage.io import imread, imshow
from skimage.transform import resize

for class_name in sorted(os.listdir(TRAINING_DIR)):
    classes.append(class_name)
    features[class_name] = {}
    class_dir = os.path.join(TRAINING_DIR, class_name)
    for file_name in sorted(os.listdir(class_dir)):
        filename = os.path.join(class_dir, file_name)
#        kp, des = ORB.compute_file(filename)
        img = imread(filename,as_gray=True)
        resized_img = resize(img, (128,64))
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=False)
        # hog_image,fd = HOG(filename)
        
        if hog_image is not None:
            # hog_image = hog_image[:32]
            features[class_name][file_name] = hog_image
            all_descriptor.extend(hog_image)

kmeans = KMeans(100)
kmeans.fit(all_descriptor)
centers = kmeans.cluster_centers_


from feature_quantization.bag_of_visual_words import get_histogram

histograms = {}
for c_name in features:
    histograms[c_name] = []
    for img in features[c_name]:
        histograms[c_name].append(get_histogram(features[c_name][img], centers))
        
X = []
Y = []
for c in histograms:
    X.extend(histograms[c])
    Y.extend([c] * len(histograms[c]))
    
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
# classifier.fit(X, Y)

test_classes = []
test_features = {}

testX = []
testY = []

for class_name in sorted(os.listdir(TEST_DIR)):
    class_dir = os.path.join(TEST_DIR, class_name)
    for file_name in sorted(os.listdir(class_dir)):
        filename = os.path.join(class_dir, file_name)
        img = imread(filename,as_gray=True)
        resized_img = resize(img, (128,64))
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True, multichannel=False)
        if hog_image is not None:
            # hog_image = hog_image[:32]
            histogram = get_histogram(hog_image, centers)
            testX.append(histogram)
            testY.append(class_name)

classifier.score(testX, testY)


from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

le = preprocessing.LabelEncoder()
training_labels_encoded = le.fit_transform(Y)
testing_labels_encoded = le.transform(testY)
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))

#TRAINING RANDOM FOREST AND RUNING IT ON TEST DATA TO MAKE PREDICTIONS
classifier.fit(X, training_labels_encoded)
predictions = classifier.predict(testX)
target_names = ['Faces','airplanes','anchor','background','barrel','camera','car_side','dalmatian','ferry','headphone','lamp','pizza','pyramid','snoopy','soccer_ball','stop_sign','strawberry','sunflower','water_lilly','windsor_chair','yin_yang']

#EVALUATION PART WHICH CONSISTS OF MEAN F1-SCORE, PER-CLASS (F1-SCORE, PRECISION, RECALL) AND CONFUSION MATRIX
print(classification_report(testing_labels_encoded, predictions, target_names=target_names,zero_division=1))
cm = confusion_matrix(testing_labels_encoded, predictions, labels=values)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = keys)
disp.plot()
plt.show()


